In [1]:
from pprint import pprint
from lxml import html
import requests
import re
import datetime
import pandas as pd

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36'}

result = []
def lenta_news(url):
    try:
        # result = []
        main_url = 'https://lenta.ru'
        response = requests.get(url, headers=header)
        root = html.fromstring(response.text)

        names = root.xpath("//div[@class = 'span8 js-main__content']//div[@class='item']")
        for el in names:
            items = {}
            url = el.xpath(".//a[@href]/@href")
            title = el.xpath(".//a[@href]/text()")
            date = el.xpath(".//a[@href]/time/@title")
            items['url'] = main_url + url[0]
            items['title'] = title[0].replace(u'\xa0', ' ')
            items['date'] = date[0]
            items['source'] = 'lenta.ru'
            result.append(items)

        return result
    except:
        print('Ошибка запроса')

def yandex_news(url):
    try:
        # result = []
        main_url = 'https://yandex.ru'
        response = requests.get(url, headers=header)
        root = html.fromstring(response.text)
        topics_info = root.xpath("//td[@class = 'stories-set__item']")

        for el in topics_info:
            items = {}
            url = el.xpath(".//a[contains(@class, 'link_theme_black')]/@href")
            title = el.xpath(".//a[contains(@class, 'link_theme_black')]/text()")
            date_full = el.xpath(".//div[@class = 'story__date']/text()")[0].replace(u'\xa0', ' ')
            date = el.xpath(".//div[@class = 'story__date']/text()")

            if re.search('вчера', date_full):
                date = str((datetime.date.today() - datetime.timedelta(days=1)))+ " " + date[0].replace(u'\xa0', ' ')[-5:]
                source = date_full[:-14]
            else:
                date = str(datetime.date.today())+ " " + date[0].replace(u'\xa0', ' ')[-5:]
                source = date_full[:-6]

            items['url'] = main_url + url[0]
            items['title'] = title[0].replace(u'\xa0', ' ')
            # items['date_full'] = date_full
            items['date'] = date
            items['source'] = source
            result.append(items)
        return result
    except:
        print('Ошибка запроса')

def mail_news(url):
    try:
        # result = []
        main_url = 'https://news.mail.ru'
        response = requests.get(url, headers=header)
        root = html.fromstring(response.text)
        topics_info = root.xpath("//div[@class = 'js-module']//li[contains(@class, 'list__item')]//a[@class = 'list__text']")

        for el in topics_info:
            items = {}
            url = el.xpath("./@href")
            for item in url:
                if re.search('https', item):
                    deep_response = requests.get(item, headers=header)
                    deep_root = html.fromstring(deep_response.text)
                    source = deep_root.xpath(".//a[@class = 'link color_gray breadcrumbs__link']//text()")
                    date = deep_root.xpath("//span[@class = 'note__text breadcrumbs__text js-ago'] / @ datetime")
                else:
                    deep_response = requests.get(main_url + item, headers=header)
                    deep_root = html.fromstring(deep_response.text)
                    source = deep_root.xpath(".//a[@class = 'link color_gray breadcrumbs__link']//text()")
                    date = deep_root.xpath("//span[@class = 'note__text breadcrumbs__text js-ago'] / @ datetime")
            title = el.xpath("./text()")
            items['url'] = main_url + url[0]
            items['title'] = title[0].replace(u'\xa0', ' ')
            items['date'] = date[0]
            items['source'] = source[0]
            result.append(items)
        return result
    except:
        print('Ошибка запроса')

lenta_news = lenta_news('https://lenta.ru/')
yandex_news = yandex_news('https://yandex.ru/news/')
mail_news = mail_news('https://news.mail.ru/')

# pprint((mail_news))




from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['news_db']
news = db.news

news.insert_many(mail_news)

In [2]:
news = db.news #selecting the coll1 in myDatabase
for document in news.find():
    print(document)


{'_id': ObjectId('5e4fb50ae25ad51fa78bc905'), 'url': 'https://lenta.ruhttps://moslenta.ru/news/pogoda-v-moskve-obnovila-chetvertyi-rekord-za-nedelyu-21-02-2020.htm/?utm_source=from_lenta', 'title': 'Погода в Москве обновила четвертый рекорд за неделю ', 'date': '21 февраля 2020', 'source': 'lenta.ru'}
{'_id': ObjectId('5e4fb50ae25ad51fa78bc906'), 'url': 'https://lenta.ru/news/2020/02/21/strax/', 'title': 'Кремль отреагировал на исследование о страхе россиян из-за ухода Путина', 'date': '21 февраля 2020', 'source': 'lenta.ru'}
{'_id': ObjectId('5e4fb50ae25ad51fa78bc907'), 'url': 'https://lenta.ru/news/2020/02/21/till/', 'title': 'Заступившуюся за девушек из порноклипа солиста Rammstein феминистку затравили', 'date': '21 февраля 2020', 'source': 'lenta.ru'}
{'_id': ObjectId('5e4fb50ae25ad51fa78bc908'), 'url': 'https://lenta.ru/news/2020/02/21/favpoezd/', 'title': 'Названы самые безопасные места в российских поездах', 'date': '21 февраля 2020', 'source': 'lenta.ru'}
{'_id': ObjectId('5e4f

In [3]:
import pandas as pd
import json
import bson
from bson import Binary, Code
from bson.json_util import dumps

In [4]:
news = db.news #selecting the coll1 in myDatabase
filter = {"type": "something"}

documents_count = db['news'].estimated_document_count()
print(documents_count)

with open("collection.json", "w") as file:
    file.write('[')

    for i, document in enumerate(news.find(), 1):
        file.write(json.dumps(document, default = str))
        if i != documents_count:
            file.write(',')
    file.write(']')
   

81


In [5]:
news_pd = pd.read_json(r'collection.json')
news_pd

,_id,date,source,title,url
0,5e4fb50ae25ad51fa78bc905,21 февраля 2020,lenta.ru,Погода в Москве обновила четвертый рекорд за н...,https://lenta.ruhttps://moslenta.ru/news/pogod...
1,5e4fb50ae25ad51fa78bc906,21 февраля 2020,lenta.ru,Кремль отреагировал на исследование о страхе р...,https://lenta.ru/news/2020/02/21/strax/
2,5e4fb50ae25ad51fa78bc907,21 февраля 2020,lenta.ru,Заступившуюся за девушек из порноклипа солиста...,https://lenta.ru/news/2020/02/21/till/
3,5e4fb50ae25ad51fa78bc908,21 февраля 2020,lenta.ru,Названы самые безопасные места в российских по...,https://lenta.ru/news/2020/02/21/favpoezd/
4,5e4fb50ae25ad51fa78bc909,21 февраля 2020,lenta.ru,Кремль оценил угрозу военного столкновения Рос...,https://lenta.ru/news/2020/02/21/peskov/
5,5e4fb50ae25ad51fa78bc90a,21 февраля 2020,lenta.ru,Совет дедушки принес внучке неожиданное богатство,https://lenta.ru/news/2020/02/21/grandpa/
6,5e4fb50ae25ad51fa78bc90b,21 февраля 2020,lenta.ru,Раскрыто отношение украинцев к Евромайдану,https://lenta.ru/news/2020/02/21/euromaidan/
7,5e4fb50ae25ad51fa78bc90c,21 февраля 2020,lenta.ru,На атомном уровне зафиксировали необъяснимое я...,https://lenta.ru/news/2020/02/21/atom/
8,5e4fb50ae25ad51fa78bc90d,21 февраля 2020,lenta.ru,Норвежский лыжник обвинил Большунова в победе ...,https://lenta.ru/news/2020/02/21/kruger/
9,5e4fb50ae25ad51fa78bc90e,2020-02-21 15:16,Газета.Ru,Турция исключила военный конфликт с Россией в ...,https://yandex.ru/news/story/Turciya_isklyuchi...
